In [4]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
batchSize=1
imageSize=[512,512]

In [2]:
print(torchvision.__version__)
print(torch.__version__)

0.14.0
1.13.0


In [7]:
gpu = torch.device("cuda")
cpu = torch.device("cpu")

In [14]:
import os
import shutil
def demo(path):
    img = [path+"/sample/"+img for img in os.listdir(path+"/sample/")]
    images=cv2.imread(img[0])
    images = cv2.resize(images, imageSize, cv2.INTER_LINEAR)
    images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
    images=images.swapaxes(1, 3).swapaxes(2, 3)
    images = list(image.to(gpu) for image in images)
    
    model_dir = path+"/result/model"
    output_image = path+"/result/test/"
    
    if os.path.exists(output_image):
        shutil.rmtree(output_image)

    os.mkdir(output_image)
    for model_path in os.listdir(model_dir):
        model=torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True) 
        in_features = model.roi_heads.box_predictor.cls_score.in_features 
        model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes=2)
        model.load_state_dict(torch.load(os.path.join(model_dir, model_path)))
        model.to(gpu)# move model to the right device
        model.eval()

        with torch.no_grad():
            pred = model(images)
        im= images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.uint8)
        im2 = im.copy()
        for i in range(len(pred[0]['masks'])):
            msk=pred[0]['masks'][i,0].detach().cpu().numpy()
            scr=pred[0]['scores'][i].detach().cpu().numpy()
            if scr>0.8 :
                im2[:,:,0][msk>0.5] = random.randint(0,255)
                im2[:, :, 1][msk > 0.5] = random.randint(0,255)
                im2[:, :, 2][msk > 0.5] = random.randint(0, 255)
        # cv2.imshow(str(scr), np.hstack([im,im2])
        cv2.imwrite(os.path.join(output_image,model_path.replace(".torch", ".png")), im2)
    

In [ ]:
import os
import shutil
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_Weights
def demoV2(path):
    img = [path+"/sample/"+img for img in os.listdir(path+"/sample/")]
    images=cv2.imread(img[0])
    images = cv2.resize(images, imageSize, cv2.INTER_LINEAR)
    images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
    images=images.swapaxes(1, 3).swapaxes(2, 3)
    images = list(image.to(gpu) for image in images)
    
    model_dir = path+"/result/modelV2"
    output_image = path+"/result/testV2/"
    
    if os.path.exists(output_image):
        shutil.rmtree(output_image)

    os.mkdir(output_image)
    for model_path in os.listdir(model_dir):
        model=torchvision.models.detection.maskrcnn_resnet50_fpn_v2(pretrained=True) 
        in_features = model.roi_heads.box_predictor.cls_score.in_features 
        model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes=2)
        model.load_state_dict(torch.load(os.path.join(model_dir, model_path)))
        model.to(gpu)# move model to the right device
        model.eval()

        with torch.no_grad():
            pred = model(images)
        im= images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.uint8)
        im2 = im.copy()
        for i in range(len(pred[0]['masks'])):
            msk=pred[0]['masks'][i,0].detach().cpu().numpy()
            scr=pred[0]['scores'][i].detach().cpu().numpy()
            if scr>0.8 :
                im2[:,:,0][msk>0.5] = random.randint(0,255)
                im2[:, :, 1][msk > 0.5] = random.randint(0,255)
                im2[:, :, 2][msk > 0.5] = random.randint(0, 255)
        # cv2.imshow(str(scr), np.hstack([im,im2])
        cv2.imwrite(os.path.join(output_image,model_path.replace(".torch", ".png")), im2)
    

In [16]:
data_dir = ["resize_128"]



In [18]:
demo(data_dir[0])

In [ ]:
demoV2(data_dir[0])